# Chemoinformatics: Molecular Graphs

## Preparation

In [ ]:
pip install rdkit networkx matplotlib

In [ ]:
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

## Build a Molecular Graph

### Build a Simple Molecular Graph

In [ ]:
# 1. Define the molecule using its SMILES string
aspirin_smiles = "CC(=O)OC1=CC=CC=C1C(=O)O"
mol = Chem.MolFromSmiles(aspirin_smiles)

# Add hydrogen atoms to the molecule for a complete graph
mol = Chem.AddHs(mol)

# 2. Create a NetworkX graph
G = nx.Graph()

# 3. Add nodes (atoms) to the graph
for atom in mol.GetAtoms():
    G.add_node(atom.GetIdx(),
               atomic_symbol=atom.GetSymbol(),
               atomic_num=atom.GetAtomicNum())

# 4. Add edges (bonds) to the graph
for bond in mol.GetBonds():
    G.add_edge(bond.GetBeginAtomIdx(),
               bond.GetEndAtomIdx(),
               bond_type=bond.GetBondType())

In [ ]:
# Visualize the graph
# Get labels for nodes (atomic symbols)
labels = nx.get_node_attributes(G, 'atomic_symbol')
# Set node positions using a layout algorithm for better visualization
pos = nx.spring_layout(G, seed=42)

plt.figure(figsize=(10, 7))
nx.draw(G, pos, labels=labels, with_labels=True, node_color='lightblue',
        node_size=500, font_size=10, font_weight='bold', edge_color='gray')

plt.title("Molecular Graph of Aspirin", fontsize=16)
plt.show()

### Going Further: Creating a Weighted Graph

We can easily extend this to create a weighted graph where the edge weight corresponds to the bond order (single, double, aromatic, etc.).

In [ ]:
# (Code from steps 1-3 is the same)
mol_weighted = Chem.MolFromSmiles(aspirin_smiles)
G_weighted = nx.Graph()

for atom in mol_weighted.GetAtoms():
    G_weighted.add_node(atom.GetIdx(), atomic_symbol=atom.GetSymbol())

# Add weighted edges
for bond in mol_weighted.GetBonds():
    # Get the bond type as a number (e.g., 1.0 for SINGLE, 2.0 for DOUBLE)
    bond_order = bond.GetBondTypeAsDouble()
    G_weighted.add_edge(bond.GetBeginAtomIdx(),
                        bond.GetEndAtomIdx(),
                        weight=bond_order)

In [ ]:
# Visualize with edge labels showing the weight
pos = nx.spring_layout(G_weighted, seed=42)
node_labels = nx.get_node_attributes(G_weighted, 'atomic_symbol')
edge_labels = nx.get_edge_attributes(G_weighted, 'weight')

plt.figure(figsize=(10, 7))
nx.draw(G_weighted, pos, labels=node_labels, with_labels=True, node_color='lightgreen', node_size=500)
nx.draw_networkx_edge_labels(G_weighted, pos, edge_labels=edge_labels)
plt.title("Weighted Molecular Graph of Aspirin (Bond Order)", fontsize=16)
plt.show()

### Exercise 1: Analyze a Different Molecule

Repeat the graph construction and visualization process for caffeine. How does its graph structure differ from aspirin's?

Hint: you can find out the SMILES String for caffeine using e.g. Wikipedia

#### 🔍 Solution (Hidden - Expand to see the answer)

<details>
<summary>Click here to see the solution after you've tried exploring the functions</summary>

Caffeine SMILES: CN1C=NC2=C1C(=O)N(C(=O)N2C)C

```import rdkit
from rdkit import Chem
import networkx as nx
import matplotlib.pyplot as plt

# 1. Define the caffeine molecule using its SMILES string
caffeine_smiles = "CN1C=NC2=C1C(=O)N(C(=O)N2C)C"
mol = Chem.MolFromSmiles(caffeine_smiles)
mol = Chem.AddHs(mol) # Add hydrogens for a complete graph

# 2. Create a NetworkX graph
G_caffeine = nx.Graph()

# 3. Add nodes (atoms) to the graph
for atom in mol.GetAtoms():
    G_caffeine.add_node(atom.GetIdx(),
                        atomic_symbol=atom.GetSymbol())

# 4. Add edges (bonds) to the graph
for bond in mol.GetBonds():
    G_caffeine.add_edge(bond.GetBeginAtomIdx(),
                        bond.GetEndAtomIdx())

# 5. Visualize the graph
labels = nx.get_node_attributes(G_caffeine, 'atomic_symbol')
pos = nx.spring_layout(G_caffeine, seed=42)

plt.figure(figsize=(10, 7))
nx.draw(G_caffeine, pos, labels=labels, with_labels=True, node_color='lightcoral',
        node_size=500, font_size=10, font_weight='bold', edge_color='gray')

plt.title("Molecular Graph of Caffeine", fontsize=16)
plt.show()
```

</details>

### Exercise 2: Color Nodes by Element

Modify the visualization code to color the nodes based on their element. This is standard practice for chemical drawings.

Hint: Create a color map dictionary (e.g., {'C': 'gray', 'O': 'red', 'N': 'blue'}). Then, create a list of colors corresponding to the nodes in your graph and pass it to the node_color argument in nx.draw().

#### 🔍 Solution (Hidden - Expand to see the answer)

<details>
<summary>Click here to see the solution after you've tried exploring the functions</summary>

```
# Create the aspirin molecule and graph as before
aspirin_smiles = "CC(=O)OC1=CC=CC=C1C(=O)O"
mol = Chem.MolFromSmiles(aspirin_smiles)
G_aspirin = nx.Graph()

for atom in mol.GetAtoms():
    G_aspirin.add_node(atom.GetIdx(), atomic_symbol=atom.GetSymbol())

for bond in mol.GetBonds():
    G_aspirin.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx())

# --- Solution for Exercise 2 ---
# 1. Define a color map for common elements
color_map = {'C': '#606060', 'O': '#E60000', 'N': '#0082E6', 'H': '#CCCCCC'}

# 2. Create a list of colors for each node in the graph
node_colors = []
for node in G_aspirin.nodes(data=True):
    symbol = node[1]['atomic_symbol']
    node_colors.append(color_map.get(symbol, 'pink')) # Default to pink if element not in map

# 3. Visualize the graph with the specified node colors
labels = nx.get_node_attributes(G_aspirin, 'atomic_symbol')
pos = nx.spring_layout(G_aspirin, seed=42)

plt.figure(figsize=(10, 7))
# Use the node_colors list for the node_color argument
nx.draw(G_aspirin, pos, labels=labels, with_labels=True, node_color=node_colors,
        node_size=500, font_size=10, font_weight='bold', edge_color='gray',
        edgecolors='black') # Add a black edge to nodes for clarity

plt.title("Aspirin Graph with Nodes Colored by Element", fontsize=16)
plt.show()
```

</details>

## Calculate Graph Metrics

Using the generated NetworkX graph for aspirin, we can calculate and print the following:

The degree of each atom (which corresponds to its valency or number of connections).

The shortest path between the two oxygen atoms in the carboxylic acid group (-C(=O)O).

In [ ]:
# Create the aspirin molecule and graph (without hydrogens for simplicity)
aspirin_smiles = "CC(=O)OC1=CC=CC=C1C(=O)O"
mol = Chem.MolFromSmiles(aspirin_smiles)
G_aspirin_heavy = nx.Graph()

for atom in mol.GetAtoms():
    G_aspirin_heavy.add_node(atom.GetIdx(), atomic_symbol=atom.GetSymbol())
for bond in mol.GetBonds():
    G_aspirin_heavy.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx())

# Calculate and print the degree of each atom (valency)
print("--- Atom Degrees (Valency) ---")
for node_idx, degree in G_aspirin_heavy.degree():
    symbol = G_aspirin_heavy.nodes[node_idx]['atomic_symbol']
    print(f"Atom {node_idx} ({symbol}): Degree = {degree}")

# Find the shortest path between the two carboxylic acid oxygens
print("\n--- Shortest Path Calculation ---")
# Find the indices of the carboxylic acid atoms: C, O, O
# The C is bonded to the ring and to two Os.
cooh_atoms = {}
for atom in mol.GetAtoms():
    if atom.GetSymbol() == 'C':
        # Check if this C is part of a carboxylic acid
        o_neighbors = [n for n in atom.GetNeighbors() if n.GetSymbol() == 'O']
        if len(o_neighbors) == 2:
            cooh_atoms['C'] = atom.GetIdx()
            cooh_atoms['O1'] = o_neighbors[0].GetIdx()
            cooh_atoms['O2'] = o_neighbors[1].GetIdx()
            break # Found our group

if cooh_atoms:
    o1_idx, o2_idx = cooh_atoms['O1'], cooh_atoms['O2']

    # Calculate shortest path using NetworkX
    path = nx.shortest_path(G_aspirin_heavy, source=o1_idx, target=o2_idx)

    # Format path for printing
    path_symbols = [G_aspirin_heavy.nodes[i]['atomic_symbol'] for i in path]
    path_str = " -> ".join([f"{idx}({sym})" for idx, sym in zip(path, path_symbols)])

    print(f"The carboxylic acid oxygens are at indices: {o1_idx} and {o2_idx}")
    print(f"The shortest path between them is: {path_str}")
    print(f"Path length: {len(path) - 1} bonds")
else:
    print("Could not identify the carboxylic acid group automatically.")

This script will perform the subgraph isomorphism test for each molecule in the database and then generate an image highlighting the matches.



## Subgraph Isomorphism Task

In the following, we will perform a subgraph isomorphism test for each molecule in a database and then generate an image highlighting the matches.

**The Problem: Finding a Pharmacophore**

A pharmacophore is a part of a molecule's structure that is responsible for its biological activity. A common and important pharmacophore is the phenylsulfonamide group.

**The Task**:

We write a Python script to search a small "database" of drug molecules and identify which ones contain the phenylsulfonamide substructure. For each match, we will visualize the molecule with the pharmacophore highlighted.

**Setup:**

Query Graph: The phenylsulfonamide substructure.

We'll represent this using a SMARTS string: c1ccc(S(=O)(=O)N)cc1.

"Database": A list of three molecules represented by their SMILES strings.

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw

# The query substructure (phenylsulfonamide) as a SMARTS string
query_smarts = "c1ccc(S(=O)(=O)N)cc1"

# Convert the SMARTS string into a query molecule object
query_mol = Chem.MolFromSmarts(query_smarts)
query_mol.SetProp("_Name", "Query: Phenylsulfonamide")

print("--- Visualizing the Query Substructure ---")

# Draw the molecule. MolsToGridImage is used here for consistency,
# but Draw.MolToImage would also work for a single molecule.
img = Draw.MolsToGridImage(
    [query_mol],
    molsPerRow=1,
    subImgSize=(300, 300),
    legends=[query_mol.GetProp("_Name")] # Use the property as a legend
)

# Display the image in Jupyter by making it the last line of the cell
img

In [ ]:
# Setup
molecule_database = {
    "Sulfanilamide": "C1=CC(=CC=C1S(=O)(=O)N)N",
    "Celecoxib": "CC1=CC=C(C=C1)C2=C(C(=NN2)C(F)(F)F)C3=CC=C(C=C3)S(=O)(=O)N",
    "Aspirin": "CC(=O)OC1=CC=CC=C1C(=O)O"
}

print(f"Searching for the phenylsulfonamide substructure...\n")

# Perform the Isomorphism Test
matching_molecules = []
matching_indices = []
molecule_names = []

for name, smiles in molecule_database.items():
    mol = Chem.MolFromSmiles(smiles)
    if mol.HasSubstructMatch(query_mol):
        print(f"✔️ Match found in: {name}")
        match_idx = mol.GetSubstructMatch(query_mol)
        matching_molecules.append(mol)
        matching_indices.append(match_idx)
        molecule_names.append(name)
    else:
        print(f"❌ No match in: {name}")

# Create the Image Object
if matching_molecules:
    img = Draw.MolsToGridImage(
        matching_molecules,
        molsPerRow=2,
        subImgSize=(300, 300),
        legends=molecule_names,
        highlightAtomLists=matching_indices
    )
    print("\nImage object created. Display it by running the 'img' variable in the next cell.")

In [ ]:
img

**Subgraph Isomorphism**: This exercise solves the subgraph isomorphism problem, which is far more common in chemistry than strict graph isomorphism. We aren't asking "Is this molecule identical to our query?" but rather "Does this molecule contain our query's structure?"

**SMILES vs. SMARTS**: We used SMILES to represent specific, concrete molecules. We used SMARTS to define the query, as it's a language designed to represent abstract structural patterns. For example, c in SMARTS matches any aromatic carbon, making it more general than C in SMILES.

**The VF2 Algorithm**: RDKit's substructure matching is powered by a highly efficient algorithm, typically a variation of the VF2 algorithm, which is designed to solve graph isomorphism problems much faster than a brute-force approach.

## Analyzing Drug-Target Networks with Bipartite Graphs

In chemoinformatics and pharmacology, we often deal with "two-mode" networks where one set of items (like drugs) interacts with another set of items (like protein targets). A simple graph isn't the right tool, because we're not modeling drug-drug or target-target interactions directly.

A bipartite graph is the perfect model for this. It's a graph whose nodes can be divided into two distinct sets, and edges only connect nodes from different sets.





In the following figure, the interaction graph (yellow) represents a bipartite graph of the two distinct protein and ligand graphs.

<img src="https://www.biorxiv.org/content/biorxiv/early/2022/08/21/2022.08.19.504617/F1.large.jpg?width=800&height=600&carousel=1" width="50%">

From: Xia, C., Feng, S.-H., Xia, Y., Pan, X., & Shen, H.-B. (2022). Leveraging Scaffold Information to Predict Protein-ligand Binding Affinity with an Empirical Graph Neural Network. Cold Spring Harbor Laboratory. https://doi.org/10.1101/2022.08.19.504617

In our case:

Set U: Drugs

Set V: Protein Targets

Edges: Represent a known interaction (e.g., a drug binds to a target).

We will investigate how to build and analyze such a network to uncover interesting relationships.

In [ ]:
# Let's create a small, illustrative dataset of drug-target interactions.
# In a real-world scenario, this would come from a database like DrugBank or ChEMBL.
data = {
    'drug': ['Drug A', 'Drug B', 'Drug C', 'Drug D', 'Drug E'],
    'target': ['EGFR', 'EGFR', 'HER2', 'CDK2', 'CDK2']
}
# Add some polypharmacology (drugs hitting multiple targets)
data['drug'].extend(['Drug B', 'Drug D'])
data['target'].extend(['HER2', 'VEGFR'])

interaction_df = pd.DataFrame(data)
print("Our Drug-Target Interaction Data:")
print(interaction_df)

### Building and Visualizing the Bipartite Graph

Now, we'll create a networkx graph. It's crucial that we tell networkx which nodes are drugs and which are targets. We do this by adding a bipartite attribute to each node.

In [ ]:
# Create an empty graph
B = nx.Graph()

# Get the unique drugs and targets
drugs = interaction_df['drug'].unique()
targets = interaction_df['target'].unique()

# Add nodes with the 'bipartite' attribute
B.add_nodes_from(drugs, bipartite=0) # 0 for drugs
B.add_nodes_from(targets, bipartite=1) # 1 for targets

# Add edges from the dataframe
B.add_edges_from(interaction_df.values)

# --- Visualize the Bipartite Graph ---
# A standard layout won't work well. We need a specific bipartite layout.
plt.figure(figsize=(10, 7))

# The bipartite_layout positions nodes in two straight lines
pos = nx.bipartite_layout(B, nodes=drugs)

# Draw the graph
nx.draw(B, pos, with_labels=True, node_size=1000, font_weight='bold',
        # Color nodes based on their set
        node_color=['skyblue' if B.nodes[node]['bipartite']==0 else 'lightgreen' for node in B.nodes])

plt.title("Drug-Target Bipartite Network", fontsize=16)
plt.show()

### Basic Analysis - Who Are the Hubs?

A simple but powerful analysis is to check the degree of each node.

A drug with a high degree is "promiscuous" or polypharmacological—it hits many targets.

A target with a high degree is a "popular" target, hit by many different drugs.

In [ ]:
# Calculate degrees for each node
degrees = dict(B.degree())

# Separate drug and target degrees for clarity
drug_degrees = {node: deg for node, deg in degrees.items() if node in drugs}
target_degrees = {node: deg for node, deg in degrees.items() if node in targets}

print("--- Drug Promiscuity (Degree) ---")
# Sort by degree for easier interpretation
print(sorted(drug_degrees.items(), key=lambda item: item[1], reverse=True))

print("\n--- Target Popularity (Degree) ---")
print(sorted(target_degrees.items(), key=lambda item: item[1], reverse=True))

#### Interpretation:

From the output, we can draw clear conclusions:

Most Promiscuous Drugs: Drug B and Drug D are tied as the most promiscuous drugs in our network, as each targets 2 different proteins.

Most Popular Targets: EGFR, HER2, and CDK2 are tied as the most popular targets. Each of them is targeted by 2 different drugs, making them central hubs in this network.

### Advanced Analysis - Bipartite Projections

This is where the real power of bipartite analysis comes in.

We can "project" the graph onto one of the node sets to uncover hidden relationships.

#### Drug-Similarity Network

Let's create a new graph where the nodes are only drugs.

An edge will exist between two drugs if they share at least one common target.

In [ ]:
# Project the bipartite graph onto the drug nodes
drug_projection = nx.bipartite.projected_graph(B, nodes=drugs)

# Visualize the drug-similarity network
plt.figure(figsize=(8, 6))
pos_drugs = nx.spring_layout(drug_projection, seed=42)
nx.draw(drug_projection, pos_drugs, with_labels=True, node_size=1000,
        node_color='skyblue', font_weight='bold')
plt.title("Projected Drug-Similarity Network", fontsize=16)
plt.show()

#### Interpretation:

We see an edge between Drug A and Drug B. Why?

<details>
<summary>Click here to see the solution.</summary>
Because they both target EGFR. This suggests they might have a similar mechanism of action or could be studied together. The same logic applies to Drug C and Drug B (sharing HER2).
</details>

### Target-Relationship Network

Now, let's do the opposite.

We'll create a graph where the nodes are only targets.

An edge will exist between two targets if they are hit by the same drug.

In [ ]:
# Project the bipartite graph onto the target nodes
target_projection = nx.bipartite.projected_graph(B, nodes=targets)

# Visualize the target-relationship network
plt.figure(figsize=(8, 6))
pos_targets = nx.spring_layout(target_projection, seed=42)
nx.draw(target_projection, pos_targets, with_labels=True, node_size=1000,
        node_color='lightgreen', font_weight='bold')
plt.title("Projected Target-Relationship Network", fontsize=16)
plt.show()

#### Interpretation:

We see an edge between EGFR and HER2. Why?

<details>
<summary>Click here to see the solution.</summary>
Because they are both targeted by Drug B.

In reality, EGFR and HER2 are part of the same family of receptor tyrosine kinases, so this relationship is chemically and biologically meaningful.

This type of analysis can help uncover potential pathway connections or suggest that drugs hitting one target might also affect the other.
</details>